In [1]:
from copy import deepcopy
from dataclasses import dataclass
from pathlib import Path
from typing import Iterable

from aoc.decorators import timeit

data_file = Path("../Data/day9.txt").read_text()

EXAMPLE = "12345"
EXAMPLE2 = "2333133121414131402"


@dataclass
class DiskItem:
    id: int
    blocks: int
    free_space: int


def make_disk_map(items: list[int]) -> list[DiskItem]:
    buffer: list[int] = []
    chunks: list[DiskItem] = []
    chunk_size = 2
    for i, item in enumerate(items):
        buffer.append(item)
        if (i % chunk_size) == (chunk_size - 1):
            chunks.append(DiskItem(id=len(chunks), blocks=buffer[0], free_space=item))
            buffer = []

    if len(buffer) > 0:
        chunks.append(DiskItem(id=len(chunks), blocks=buffer[0], free_space=0))

    return chunks


def make_initial_map(disk_map: list[DiskItem]):
    initial_map: list[str] = []
    for disk_item in disk_map:
        initial_map += ([str(disk_item.id)] * disk_item.blocks) + (
            ["."] * disk_item.free_space
        )

    return initial_map


def prepare(input: str):
    return make_disk_map(list(map(int, input.splitlines()[0])))


def print_line(map_line: list[str]):
    print("".join(map_line))


def find_fitting_spot(map_line: list[str], block_size: int):
    new_map = deepcopy(map_line)
    new_map_count = len(new_map)
    start = 0
    while True:
        if start >= new_map_count:
            return

        start_item = new_map[start]
        if start_item != ".":
            start += 1
            continue

        if block_size == 1:
            return start, start

        end = start + 1
        if end >= new_map_count:
            return

        end_item = new_map[end]
        if end_item != ".":
            start = end + 1
            continue

        if block_size == 2:
            return start, end

        empty_space = 2
        while block_size > empty_space:
            end += 1
            if end >= new_map_count:
                return

            end_item = new_map[end]
            if end_item != ".":
                break

            empty_space += 1

        if block_size > empty_space:
            start = end + 1
            continue

        return start, end


def compress_disk(initial_map: list[str], disk_map: list[DiskItem], *, debug=False):
    new_map = deepcopy(initial_map)
    last_swapped_id: int | None = None
    for i in reversed(range(len(new_map))):
        current_item = new_map[i]
        if current_item == ".":
            continue

        disk_item = disk_map[int(current_item)]
        if last_swapped_id == disk_item.id:
            continue

        empty_spot = find_fitting_spot(new_map, disk_item.blocks)
        if empty_spot is None:
            continue

        empty_start, empty_end = empty_spot
        free_space_range = range(empty_start, empty_end + 1)
        assert all(map(lambda index: new_map[index] == ".", free_space_range))
        assert len(free_space_range) == disk_item.blocks
        assert new_map[empty_start - 1] != "."

        # if any(map(lambda enumerated_index: enumerated_index[1] >= (i - enumerated_index[0]), enumerate(free_space_range))):
        #     return

        # Raaaaaawwwwrrrrr
        for free_space_index_index, free_space_index in enumerate(free_space_range):
            new_file_index = free_space_index
            new_empty_index = i - free_space_index_index
            if new_file_index >= new_empty_index:
                break

            new_map[new_file_index], new_map[new_empty_index] = current_item, "."

        last_swapped_id = disk_item.id

        if debug:
            print_line(new_map)

    return new_map


def make_checksum(compressed_disk: Iterable[str]):
    def map_item(enumerated_item: tuple[int, str]):
        index, item = enumerated_item
        if item == ".":
            return 0

        return index * int(item)

    products = list(map(map_item, enumerate(compressed_disk)))

    return sum(products)


# 00992111777.44.333....5555.6666.....8888.. ---- from me!
# 00992111777.44.333....5555.6666.....8888.. ---- from advent of code

In [ ]:
@timeit
def part2(input: str, *, debug=False):
    disk_map = prepare(input)
    initial_map = make_initial_map(disk_map)
    compressed_map = compress_disk(initial_map, disk_map, debug=debug)

    return make_checksum(compressed_map)


example_result = part2(EXAMPLE2, debug=True)

assert (
    example_result == 2858
), f"Expected example result to be 2858, but got {example_result} instead"

result = part2(data_file)

print("result is", result)

assert (
    result == 6421724645083
), f"Expected example result to be 6421724645083, but got {result} instead"

0099.111...2...333.44.5555.6666.777.8888..
0099.1117772...333.44.5555.6666.....8888..
0099.1117772...333.44.5555.6666.....8888..
0099.1117772...333.44.5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
def part2(input, debug): took: 0.0003 sec
def part2(input, debug): took: 172.9392 sec
result is 6421724645083


AssertionError: 